In [45]:
# Modules

In [46]:
import numpy as np
import pandas as pd

import time
import matplotlib.pyplot as plt

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

import math

import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

In [47]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

In [48]:
# Trajectories Functions

In [49]:
# Brownian Motion
def bm_2d(n_steps = 1000, speed=6, s_x_pos=0, s_y_pos=0):
  """
    Arguments:
      n_steps:
      speed:
      s_x_pos:
      s_y_pos:
    Returns:
      BM_2d_df
  """  
  s_pos = [s_x_pos,s_y_pos]

# Init velocity vector
  velocity = Vec2d(speed,0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index = True)

  for i in range(n_steps-1):
      turn_angle = np.random.uniform(low=-np.pi,high=np.pi)
      velocity = velocity.rotated(turn_angle)

      temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
      BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index = True)

  return BM_2d_df

In [50]:
#CRW
def CRW_2d(n_steps = 1000, speed=6, s_x_pos=0, s_y_pos=0, cauchy=0.7):
  """
    Arguments:
      n_steps:
      speed:
      s_x_pos:
      s_y_pos:
      cauchy:
    Returns:
      Levy_2d_df
  """  
  s_pos = [s_x_pos,s_y_pos]

# Init velocity vector
  velocity = Vec2d(speed,0)

  CRW_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index = True)

  for i in range(n_steps-1):
      turn_angle = wrapcauchy.rvs(c=cauchy,loc=0)
      velocity = velocity.rotated(turn_angle)

      temp_df = pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i]+velocity.x, 'y_pos': CRW_2d_df.y_pos[i]+velocity.y}])
      CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index = True)

  return CRW_2d_df

In [51]:
#Levy
def Levy_2d(n_steps = 1000, speed=6, s_x_pos=0, s_y_pos=0, cauchy=0.7, alpha=1,beta=1, stdms=6):
  """
    Arguments:
      n_steps:
      speed:
      s_x_pos:
      s_y_pos:
      cauchy:
      alpha:
      beta:
      stdms:
    Returns:
      Levy_2d_df
  """  
  s_pos = [s_x_pos,s_y_pos]

# Init velocity vector
  velocity = Vec2d(speed,0)

  Levy_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  Levy_2d_df = pd.concat([Levy_2d_df, temp_df], ignore_index = True)

  i=0
  while i in range(n_steps-1):

    # Get random n_steps from Levy distribution 
    step_size = levy_stable.rvs(alpha,beta,stdms)

    # round to integer number
    step_size = int(np.ceil(abs(step_size)))

    #Get turning angle from cauchy distribution
    turn_angle = wrapcauchy.rvs(c=cauchy,loc=0)
    velocity = velocity.rotated(turn_angle)

    for j in range(step_size)-1:

      temp_df = pd.DataFrame([{'x_pos': Levy_2d_df.x_pos[i]+velocity.x, 'y_pos': Levy_2d_df.y_pos[i]+velocity.y}])
      Levy_2d_df = pd.concat([Levy_2d_df, temp_df], ignore_index = True)
      i+=1

  return Levy_2d_df

In [52]:
# Complementary Functions

In [53]:
# Vector distance calculation function
def calc_vdistance(vector1, vector2):
  """
    Arguments:
      df1:
      df1:
      s_pos:
    Returns:
      vdistance
  """  
  vdistance=(np.sqrt((vector2[0]-vector1[0])**2+(vector2[1]-vector1[1])**2))
  return vdistance

In [54]:
# Angle Vector calculation function
def calc_vangle(vector1, vector2, vector3):
  """
    Arguments:
      vector1
      vector2
    Returns:
      angle
  """  
  cross=np.cross([vector1[0]-vector2[0],vector1[1]-vector2[1]],[vector3[0]-vector2[0],vector3[1]-vector2[1]])
  dot=np.dot([vector1[0]-vector2[0],vector1[1]-vector2[1]],[vector3[0]-vector2[0],vector3[1]-vector2[1]])
  angle=math.atan2(cross,dot)
  return angle

In [55]:
# Metrics Functions

In [56]:
#Compute Path length
def compute_path_length(rw_df):
  
  pl = np.array([calc_vdistance(rw_df.iloc[i-1], rw_df.iloc[i]) for i in range(1,rw_df.shape[0])])
  path_length=np.cumsum(pl)

  return path_length

In [57]:
#Compute MSD
def compute_msd(rw_df):
  
  d_vector=pos=np.array([calc_vdistance([0,0],rw_df.iloc[i]) for i in range(rw_df.shape[0])])
  MSD=np.zeros_like(d_vector)
  for i in range(1,len(d_vector)):
    MSD[i]=np.average((d_vector[i:]-d_vector[:-i])**2)

  return MSD

In [58]:
# Widgets

In [59]:
Traj_Select = pnw.RadioBoxGroup(name='Trajectory Selection', value='Brownian Motion (BM)', 
                        options=['Brownian Motion (BM)','Correlated Random Walk (CRW)','Levy Flight (LF)'])
Steps_Number = pnw.EditableIntSlider(name = 'Number of Steps', width = 200, value = 1000, step = 10, start = 0, end = 10000)
Speed_Select = pnw.EditableIntSlider(name='Speed Selection', width=200, value=6, step=1, start=0, end=25)
Cauchy_Coeff = pnw.FloatSlider(name = 'Cauchy Coefficient', width = 200, value = 0.7, step = 0.01, start = 0.01, end = 0.99)
Start_Pos_X = pnw.EditableIntSlider(name='Starting Position X', width=200, value= 0, step= 1, start=-20, end=20)
Start_Pos_Y = pnw.EditableIntSlider(name='Starting Position Y', width=200, value= 0, step= 1, start=-20, end=20)
Levy_Alpha = pnw.FloatSlider(name = 'Levy Flight Alpha', width = 200, value = 1, step = 0.01, start = 0.01, end = 1.99)
Levy_Beta = pnw.FloatSlider(name = 'Levy Flight Beta', width = 200, value = 0, step = 0.01, start = -1, end = 1)
Standard_MS = pnw.EditableFloatSlider(name='Levy SMS', width=200, value= 3, step= 0.5, start=0, end=20)
Metric_Selection = pnw.Select(name = 'Metric Selection', value='None', options=['None','Path Length','MSD'],width=200)

In [60]:
# Panel Functions

In [61]:
# Display options per trajectory panel
@pn.depends(Traj_Select)
def display_Column(Traj_Select):
  if Traj_Select=='Brownian Motion (BM)':
    column=pn.Column(Start_Pos_Y)
  elif Traj_Select=='Correlated Random Walk (CRW)': 
    column=pn.Column(Start_Pos_Y,Cauchy_Coeff)
  elif Traj_Select=='Levy Flight (LF)':
    column=pn.Column(Start_Pos_Y,Cauchy_Coeff,Levy_Alpha,Levy_Beta,Standard_MS)
  return column

In [62]:
# Traj Plot Panel
@pn.depends(Traj_Select,Steps_Number,Speed_Select,Cauchy_Coeff,Start_Pos_X,Start_Pos_Y,Levy_Alpha,Levy_Beta,Standard_MS)
def plot_traj(Traj_Select,Steps_Number,Speed_Select,Cauchy_Coeff,
              Start_Pos_X,Start_Pos_Y,Levy_Alpha,Levy_Beta,Standard_MS):
  global rw_df
  rw_df=pd.DataFrame(columns=['x_pos','y_pos'])

  if Traj_Select=='Brownian Motion (BM)':
    rw_df=bm_2d(n_steps=Steps_Number, speed=Speed_Select, s_x_pos=Start_Pos_X, s_y_pos=Start_Pos_Y)
  elif Traj_Select=='Correlated Random Walk (CRW)': 
    rw_df=CRW_2d(n_steps=Steps_Number, speed=Speed_Select, s_x_pos=Start_Pos_X, s_y_pos=Start_Pos_Y, cauchy=Cauchy_Coeff)
  elif Traj_Select=='Levy Flight (LF)':
    rw_df=Levy_2d(n_steps=Steps_Number, speed=Speed_Select, s_x_pos=Start_Pos_X, 
                  s_y_pos=Start_Pos_Y, cauchy=Cauchy_Coeff, 
                  alpha=Levy_Alpha, beta=Levy_Beta, stdms=Standard_MS)



  fig_traj_rw=go.Figure()
  fig_traj_rw.add_trace(
      go.Scatter3d(x = rw_df.x_pos,
                   y = rw_df.y_pos,
                   z = rw_df.index,
                   marker = dict(size=2),
                   line = dict(color='red', width=2),
                   mode = 'lines'     
      )
  )
  fig_traj_rw.update_layout(width=500,height=570)
  return fig_traj_rw

In [63]:
# Display Metrics Panel
@pn.depends(Traj_Select,Steps_Number,Speed_Select,Cauchy_Coeff,Start_Pos_X,Start_Pos_Y,Levy_Alpha,Levy_Beta,Standard_MS,Metric_Selection)
def display_Metric(Traj_Select,Steps_Number,Speed_Select,Cauchy_Coeff,
              Start_Pos_X,Start_Pos_Y,Levy_Alpha,Levy_Beta,Standard_MS,Metric_Selection):
  #time.sleep(3)
  if Metric_Selection=='None':
    metric=pd.DataFrame([])
  elif Metric_Selection=='Path Length': 
   metric=compute_path_length(rw_df)
  elif Metric_Selection=='MSD':
    metric=compute_msd(rw_df)
  
  fig_metric=go.Figure()
  fig_metric.add_trace(
      go.Scatter(x = np.arange(len(metric)),
                   y = metric,
                   marker = dict(size=2),
                   line = dict(color='blue', width=2),
                   mode = 'lines'
      )
  )
  fig_metric.update_layout(height=570)

  if Metric_Selection=='None':
    return metric
  else:
    return fig_metric

In [64]:
# Main Code

In [71]:
def main():
  return pn.Row(pn.Column('# Dashboard RW \n Diego Rodriguez',
                          Traj_Select,
                          Metric_Selection,
                          Steps_Number,
                          Speed_Select,
                          Start_Pos_X,
                          display_Column,width=220),pn.Column('### Random Walk Plot',
                                                              plot_traj),pn.Column('### Metric Plot',
                                                                                   display_Metric,width=720),background='WhiteSmoke',height=650)

In [72]:
# Code Execution

In [73]:
main()

Row(background='WhiteSmoke', height=650)
    [0] Column(width=220)
        [0] Markdown(str)
        [1] RadioBoxGroup(name='Trajectory Selection', options=['Brownian Motion (BM)', ...], value='Brownian Motion (BM)')
        [2] Select(name='Metric Selection', options=['None', 'Path Length', ...], value='None', width=200)
        [3] EditableIntSlider(end=10000, name='Number of Steps', step=10, value=1000, width=200)
        [4] EditableIntSlider(end=25, name='Speed Selection', value=6, width=200)
        [5] EditableIntSlider(end=20, name='Starting Position X', start=-20, width=200)
        [6] ParamFunction(function)
    [1] Column
        [0] Markdown(str)
        [1] ParamFunction(function)
    [2] Column(width=720)
        [0] Markdown(str)
        [1] ParamFunction(function)